In [1]:
import json
import re

In [2]:
with open("credits.csv") as credits:
    print("total number of lines: " + str(sum(1 for line in credits)))

total number of lines: 45477


In [3]:
def raw_text_conv(raw_text):
    text = raw_text.replace("None", "null")
    text = re.sub(r"\"\"([^\\']*)\\'([^\\']*)\"\"", r'@@\1@@\2@@', text)
    text = re.sub(r"\"\"([^\\']*)\\'([^\\']*)\\'([^\\']*)\\'([^\\']*)\"\"", r'@@\1@@\2@@\3@@\4@@', text)
    text = re.sub("\"\"([^']*)'([^']*)\"\"", r"'\1@@\2'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3@@\4'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3@@\4@@\5'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3@@\4@@\5@@\6'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3@@\4@@\5@@\6@@\7'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3@@\4@@\5@@\6@@\7@@\8'", text)
    text = re.sub("\"\"([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)'([^']*)\"\"", r"'\1@@\2@@\3@@\4@@\5@@\6@@\7@@\8@@\9'", text)
    text = re.sub("\"\"([^\"]*)\"\"", r"@@\1@@",text)
    text = text.replace(r"\'", "@@")
    text = text.replace("\"", "@@")
    text = re.sub("'([^']*)'", r'"\1"', text)
    text = text.replace("@@", "'")
    text = text.replace("\\", "/")
    return text

In [4]:
def process_casts(raw_casts, movie_id, cast_in, cast, cast_set):
    if (len(raw_casts) < 3):
        return
    sql1 = 'INSERT INTO Cast_In(movie_id, cast_id, character) VALUES ("{}", "{}", "{}");\n'
    sql2 = 'INSERT INTO Cast(id, gender, name) VALUES ("{}", "{}", "{}");\n'
    # data cleaning
    casts = raw_text_conv(raw_casts)
    # convert the data into json format
    try:
        casts_js = json.loads(casts)
    except json.JSONDecodeError as err:
        return casts, err, raw_casts
    for single_cast in casts_js:
        # for Cast_In table movie_id cast_id, character
        cast_in.write(sql1.format(movie_id, single_cast["cast_id"], single_cast["character"]))
        # for the Cast table: name, gender, cast_id
        if single_cast["cast_id"] not in cast_set:
            cast_set.add(single_cast["cast_id"])
            try: 
                cast.write(sql2.format(single_cast["cast_id"], single_cast['gender'], single_cast['name']))
            except KeyError as e:
                print (single_cast)
                raise e


In [5]:
def process_crews(raw_crews, movie_id, crew_in, crew, crew_set):
    if (len(raw_crews) < 3):
        return
    sql1 = 'INSERT INTO Crew_In(movie_id, crew_id, job) VALUES ("{}", "{}", "{}");\n'
    sql2 = 'INSERT INTO Crew(id, gender, name) VALUES ("{}", "{}", "{}");\n'
    crews = raw_text_conv(raw_crews)
    try:
        crews_js = json.loads(crews)
    except json.JSONDecodeError as err:
        return crews, err, raw_crews
    for single_crew in crews_js:
        #  movie_id cast_id, character - 
        crew_in.write(sql1.format(movie_id, single_crew["id"], single_crew["job"]))
        # in a cast table: name, gender, id --- set
        if single_crew["id"] not in crew_set:
            crew_set.add(single_crew["id"])
            try:
                crew.write(sql2.format(single_crew["id"], single_crew['gender'], single_crew['name']))
            except KeyError as e:
                print (single_crew)
                raise e

In [6]:
cast_set = set()
crew_set = set()
cast_in = open("cast_in.sql", "w")
crew_in = open("crew_in.sql", "w")
cast = open("cast.sql", "w")
crew = open("crew.sql", "w")
line_num = 1
with open("credits.csv") as credits:
    raw_line = credits.readline()
    raw_line = credits.readline()
    while raw_line:
        line_num += 1
        line = re.split(']"?,', raw_line)
        movie_id = line[2][:-1]
        raw_casts = line[0][1:] + "]"
        raw_crews = line[1][1:] + "]"
        err = process_casts(raw_casts, movie_id, cast_in, cast, cast_set)
        if (err):
            raise err[1]
        err = process_crews(raw_crews, movie_id, crew_in, crew, crew_set)
        if (err):
            raise err[1]
        
        raw_line = credits.readline()
cast_in.close()
crew_in.close()
cast.close()
crew.close()
print("done")

done
